<a href="https://colab.research.google.com/github/LiLee1/Thesis/blob/main/AutoML_Models_for_Modulation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modulation Classification

Developing AutoML Model for AWGN Channel for Modulation Classification. SNR Ratio's(in dB) of Channel are $[−20,-18,-16,-14,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12,14,16,18]$.

**Data**: RadioML Dataset

**Modulation Classification:**
We will use AutoML to create a Classifier to Predict Labels.

## Imports

### Importing Data

Files are uploaded in Google Drive. Notebook is connected to Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive/My\ Drive/Modulation-Classification/Real\ Datasets/RadioML-Dataset/AutoML

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/modulation recognition/chapter3/AutoML

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/modulation recognition/chapter3/AutoML


In [ ]:
# !mkdir researchHub   #然后创建文件夹，自己定名字

# %cd './researchHub'   #进入你创建的这个文件夹：
# os.chdir('./researchHub') 

### Importing Libraries

We will be using AutoKeras for generating AutoML Models.
Source of Documentation: [https://autokeras.com/](https://autokeras.com/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import seaborn as sns
import scipy.io
import os
from sklearn.model_selection import train_test_split

# Importing Preprocessing Libraries
from Import_Functions import ImportData, ImportDatasets
from Plot_Results import EvaluateData, PlotResults

# AutoML Libraries
# !pip3 install autokeras
import autokeras as ak

# Tensorflow Libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, ReLU, Conv2D, Input, MaxPooling2D, BatchNormalization, AveragePooling2D, Reshape, ZeroPadding2D, Dropout
from tensorflow.keras.models import Model

ImportError: ignored

## Customised AutoML Blocks

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

import tensorflow as tf
tf.__version__

# import keras
# keras.__version__

Python 3.7.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



'2.7.0'

In [ ]:
class SqueezeLayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        input_node = inputs[0]
        output_node = tf.squeeze(input_node,axis=2)
        return output_node

class AddDimsLayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        input_node = inputs[0]
        output_node = tf.expand_dims(input_node,axis=1)
        return output_node

class PermuteLayerBlock(ak.Block):
    def build(self, hp, shape=None, inputs=None):
        input_node = inputs[0]
        dims = len(input_node.shape.as_list()[1:])
        output_node = tf.keras.layers.Permute([*range(dims,0,-1)])(input_node)
        return output_node

Callbacks

In [ ]:
EarlyStoppingCallbacks = [tf.keras.callbacks.EarlyStopping(patience=5)]

## Importing Data

In [ ]:
X_Train, y_Train, X_Valid, y_Valid = ImportDatasets("/content/drive/MyDrive/modulation recognition/chapter3/Data/RML2016.10a_dict.pkl",test_size=0.2)

## AutoML ResNet Model for RadioML Dataset
For RadioML Dataset, Input Dimensions is (2,128)

### Creating Classifier

In [ ]:
tf.keras.backend.clear_session()

input_node = ak.Input()
addDims_node = AddDimsLayerBlock()(input_node)
permute_node = PermuteLayerBlock()(addDims_node)
resnet_node = ak.ResNetBlock(version='v1')(permute_node)
dense_node = ak.DenseBlock()(resnet_node)
output_node = ak.ClassificationHead(num_classes=11,
                                    loss='categorical_crossentropy')(dense_node)

RadioMLClassifier = ak.AutoModel(inputs=input_node,
                                outputs=output_node,
                                overwrite=True,
                                tuner='greedy',
                                max_trials=3)

RadioMLClassifier.fit(X_Train, y_Train, epochs=3, batch_size=1536, validation_split=0.05,shuffle=True,callbacks=EarlyStoppingCallbacks)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
res_net_block_1...|False             |?                 
res_net_block_1...|resnet50          |?                 
res_net_block_1...|False             |?                 
dense_block_1/u...|False             |?                 
dense_block_1/n...|3                 |?                 
dense_block_1/u...|64                |?                 
dense_block_1/d...|0                 |?                 
dense_block_1/u...|128               |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Epoch 1/3
  7/110 [>.............................] - ETA: 6:47:02 - loss: 5.4786 - accuracy: 0.0268

Model Summary

In [ ]:
RadioML_Model = RadioMLClassifier.export_model()
RadioML_Model.summary()
tf.keras.backend.clear_session()
tf.keras.utils.plot_model(RadioML_Model, to_file=/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_Model_ResNet.png', show_shapes=True,show_layer_names=True)


### Training and Evaluating Model

Training the Model

In [ ]:
RadioML_Model.fit(X_Train, y_Train, epochs=25, batch_size=1536, validation_split=0.05,shuffle=True)

Evaluate the Model

In [ ]:
EvaluateData(RadioML_Model, X_Valid, y_Valid, "/content/drive/MyDrive/modulation recognition/chapter3/Results/RadioML_Accuracy_ResNet.jpg")

Save Model

In [ ]:
RadioML_Model.save("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_ResNet.h5")

## AutoML CLDNN Model for RadioML Dataset

For RadioML Dataset, Input Dimensions is (2,128)

### Creating Classifier

In [ ]:
tf.keras.backend.clear_session()

input_node = ak.Input()
addDims_node = AddDimsLayerBlock()(input_node)
permute_node = PermuteLayerBlock()(addDims_node)
conv_node = ak.ConvBlock()(permute_node)
reshape_node = SqueezeLayerBlock()(conv_node)
rnn_node = ak.RNNBlock(return_sequences=True,
                       bidirectional=True,
                       layer_type='lstm')(reshape_node)
dense_node = ak.DenseBlock()(rnn_node)
output_node = ak.ClassificationHead(num_classes=11,
                                    loss='categorical_crossentropy')(dense_node)

RadioMLClassifier = ak.AutoModel(inputs=input_node,
                                outputs=output_node,
                                overwrite=True,
                                tuner='greedy',
                                max_trials=10)

RadioMLClassifier.fit(X_Train, y_Train, epochs=10, batch_size=1024, validation_split=0.05,shuffle=True,callbacks=EarlyStoppingCallbacks)

Model Summary

In [ ]:
RadioML_Model = RadioMLClassifier.export_model()
RadioML_Model.summary()
tf.keras.backend.clear_session()
tf.keras.utils.plot_model(RadioML_Model, to_file='/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_Model_CLDNN.png', show_shapes=True,show_layer_names=True)

### Training and Evaluating Model

Training the Model

In [ ]:
RadioML_Model.fit(X_Train, y_Train, epochs=100, batch_size=1024, validation_split=0.05,shuffle=True)

Evaluate the Model

In [ ]:
EvaluateData(RadioML_Model, X_Valid, y_Valid, "/content/drive/MyDrive/modulation recognition/chapter3/Results/RadioML_Accuracy_CLDNN.jpg")

Save Model

In [ ]:
RadioML_Model.save("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_CLDNN.h5")

## AutoML CNN Model for RadioML Dataset

For RadioML Dataset, Input Dimensions is (2,128)

### Creating Classifier

In [ ]:
tf.keras.backend.clear_session()

input_node = ak.Input()
addDims_node = AddDimsLayerBlock()(input_node)
permute_node = PermuteLayerBlock()(addDims_node)
conv_node = ak.ConvBlock()(permute_node)
dense_node = ak.DenseBlock()(conv_node)
output_node = ak.ClassificationHead(num_classes=11,
                                    loss='categorical_crossentropy')(dense_node)

RadioMLClassifier = ak.AutoModel(inputs=input_node,
                                outputs=output_node,
                                overwrite=True,
                                tuner='greedy',
                                max_trials=20)

RadioMLClassifier.fit(X_Train, y_Train, epochs=10, batch_size=2048, validation_split=0.05,shuffle=True,callbacks=EarlyStoppingCallbacks)

Model Summary

In [ ]:
RadioML_Model = RadioMLClassifier.export_model()
tf.keras.backend.clear_session()
RadioML_Model.summary()
tf.keras.utils.plot_model(RadioML_Model, to_file='/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_Model_CNN.png', show_shapes=True,show_layer_names=True)

### Training and Evaluating Model

Training the Model

In [ ]:
RadioML_Model.fit(X_Train, y_Train, epochs=100, batch_size=1024, validation_split=0.05,shuffle=True)

Evaluate the Model

In [ ]:
EvaluateData(RadioML_Model, X_Valid, y_Valid, "/content/drive/MyDrive/modulation recognition/chapter3/Results/RadioML_Accuracy_CNN.jpg")

Save Model

In [ ]:
RadioML_Model.save(/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_CNN.h5")

## AutoML RNN Model for RadioML Dataset

For RadioML Dataset, Input Dimensions is (2,128)

### Creating Classifier

In [ ]:
tf.keras.backend.clear_session()

input_node = ak.Input()
permute_node = PermuteLayerBlock()(input_node)
rnn_node = ak.RNNBlock(return_sequences=True,
                       bidirectional=True,
                       layer_type='lstm')(permute_node)
dense_node = ak.DenseBlock()(rnn_node)
output_node = ak.ClassificationHead(num_classes=11,
                                    loss='categorical_crossentropy')(dense_node)

RadioMLClassifier = ak.AutoModel(inputs=input_node,
                                outputs=output_node,
                                overwrite=True,
                                tuner='greedy',
                                max_trials=20)

RadioMLClassifier.fit(X_Train, y_Train, epochs=10, batch_size=2048, validation_split=0.05,shuffle=True,callbacks=EarlyStoppingCallbacks)

Model Summary

In [ ]:
RadioML_Model = RadioMLClassifier.export_model()
RadioML_Model.summary()
tf.keras.backend.clear_session()
tf.keras.utils.plot_model(RadioML_Model, to_file='/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_Model_RNN.png', show_shapes=True,show_layer_names=True)

### Training and Evaluating Model

Training the Model

In [ ]:
RadioML_Model.fit(X_Train, y_Train, epochs=100, batch_size=1024, validation_split=0.05,shuffle=True)

Evaluate the Model

In [ ]:
EvaluateData(RadioML_Model, X_Valid, y_Valid, "/content/drive/MyDrive/modulation recognition/chapter3/Results/RadioML_Accuracy_RNN.jpg")

Save Model

In [ ]:
RadioML_Model.save("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_RNN.h5")

## Plot Results


### RadioML Model

Importing Data and Models

In [ ]:
RadioML_ResNet = tf.keras.models.load_model("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_ResNet.h5")
RadioML_CLDNN = tf.keras.models.load_model("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_CLDNN.h5")
RadioML_CNN = tf.keras.models.load_model("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_CNN.h5")
RadioML_RNN = tf.keras.models.load_model("/content/drive/MyDrive/modulation recognition/chapter3/Models/RadioML_RNN.h5")

Plotting Results

In [ ]:
Models = {"RadioML AutoML ResNet Model":RadioML_ResNet, 
          "RadioML AutoML CLDNN Model":RadioML_CLDNN,
          "RadioML AutoML CNN Model":RadioML_CNN, 
          "RadioML AtuoML RNN Model":RadioML_RNN}
PlotResults(Models, X_Valid, y_Valid, "/content/drive/MyDrive/modulation recognition/chapter3/Results/RadioML_Accuracies.jpg")